### 1. package imports

In [1]:
from pathlib import Path
import numpy as np 
import rasterio as rio
import glob
import pandas as pd
import xarray as xr
import geopandas as gpd
from shapely.geometry import box,Polygon, Point
import tqdm
from tqdm.notebook import tqdm as ntqdm
import matplotlib.pyplot as plt
from multiprocessing import Pool
import sys
curr_dir = Path.cwd()
sys.path.append(str(curr_dir.parent))
#personal imports 
# from run_smap_urgell import list_all_input_files_in_series
# from run_smap_urgell import SMAP_area_interest, process_SMAP
# from run_smos_urgell import read_smos_DBL_file
# from plot_utils import make_nice_hist


### 2. specify folders and paths

In [17]:
# INPUTS
dispatch_sm = Path("/mnt/RAID2/ACCWA/h18v04/dispatch_hybrid_21r_update") #### SMAP SM
start_time = "20230101" # format = yearmonthday = yyyymmdd
end_time = "20230301"
shapefile_path = Path("/data/PhDGiovanni/5.IrriEst/DATA/INSITU/Urgell/urgell_shape.geojson")
shapefile = gpd.read_file(shapefile_path)
### if you don't have a shapefile ready, then comment the previous 2 lines and uncomment the following lines and specify just the border extent 
# west = 0.6
# south = 41.5
# east = 1.1
# north = 41.8
# shapefile = gpd.GeoDataFrame(geometry=[box(west, south, east, north)], crs = "EPSG:4326")

# OUTPUT
outfold = Path(f"/data/PhDGiovanni/5.IrriEst/DATA/SATELLITES/test") # here a folder called "Dispatch_SMAP21r_1km.zarr" will be created

### 3. run conversion

In [18]:

fold_satellite_sm  = outfold / 'Urgell_Dispatch_SMAP' #### SMAP
fold_satellite_sm.mkdir(exist_ok=True, parents=True)
file_sm_zarr = fold_satellite_sm / 'Dispatch_SMAP21r_1km.zarr' #### SMAP
start_time = pd.Timestamp(start_time)
end_time = pd.Timestamp(end_time)

if not file_sm_zarr.exists(): # do not repeat if the file already exists
    geometries = shapefile.dissolve().to_crs(4326) # dissolve all the polygons in the shapefile
    xmin, ymin, xmax, ymax = geometries.total_bounds # get the boarder of the shapefile
    soil_moisture_files = list(dispatch_sm.glob('*/SM*_P21r.asc')) # get all the files in the folder
    # filter by dates
    soil_moisture_files = [f for f in soil_moisture_files if start_time <= pd.Timestamp(f.parent.name) <= end_time]
    soil_moisture_files.sort()
    res = []
    times = []
    for file in tqdm.tqdm(soil_moisture_files, total = len(soil_moisture_files)):
        ff  = xr.open_dataset(file,engine = 'rasterio',masked=True).rio.write_crs('EPSG:4326')
        ff2 = ff.rio.clip([box(xmin, ymin, xmax, ymax)], from_disk=True, all_touched=True)
        res.append(ff2.band_data.sel(band = 1))
        times.append(pd.Timestamp(file.parent.name))
    time = pd.Series(index = times)
    time.index.name  = 'time'
    all_xr_sm_smap = xr.concat(res, dim = time.index).to_dataset(name = 'dispatch_SM')
    all_xr_sm_smap.to_zarr(file_sm_zarr)
else:
    all_xr_sm_smap = xr.open_zarr(file_sm_zarr)

100%|██████████| 60/60 [00:27<00:00,  2.21it/s]
/home/gpaolini/miniconda3/envs/AiDNN2/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [19]:
soil_moisture_files

[PosixPath('/mnt/RAID2/ACCWA/h18v04/dispatch_hybrid_21r_update/20230101/SM1km_h18v04_20230101_P21r.asc'),
 PosixPath('/mnt/RAID2/ACCWA/h18v04/dispatch_hybrid_21r_update/20230102/SM1km_h18v04_20230102_P21r.asc'),
 PosixPath('/mnt/RAID2/ACCWA/h18v04/dispatch_hybrid_21r_update/20230103/SM1km_h18v04_20230103_P21r.asc'),
 PosixPath('/mnt/RAID2/ACCWA/h18v04/dispatch_hybrid_21r_update/20230104/SM1km_h18v04_20230104_P21r.asc'),
 PosixPath('/mnt/RAID2/ACCWA/h18v04/dispatch_hybrid_21r_update/20230105/SM1km_h18v04_20230105_P21r.asc'),
 PosixPath('/mnt/RAID2/ACCWA/h18v04/dispatch_hybrid_21r_update/20230106/SM1km_h18v04_20230106_P21r.asc'),
 PosixPath('/mnt/RAID2/ACCWA/h18v04/dispatch_hybrid_21r_update/20230107/SM1km_h18v04_20230107_P21r.asc'),
 PosixPath('/mnt/RAID2/ACCWA/h18v04/dispatch_hybrid_21r_update/20230108/SM1km_h18v04_20230108_P21r.asc'),
 PosixPath('/mnt/RAID2/ACCWA/h18v04/dispatch_hybrid_21r_update/20230109/SM1km_h18v04_20230109_P21r.asc'),
 PosixPath('/mnt/RAID2/ACCWA/h18v04/dispatch_h